In [141]:
import os
import json
import pandas as pd

In [3]:
data_path = os.path.join('..', 'data' ,'raw', 'train.json')
f = open(data_path)
data = json.load(f)
f.close()

In [91]:
test_data_path = os.path.join('..', 'data' ,'raw', 'test.json')
f = open(test_data_path)
test_data = json.load(f)
f.close()

In [4]:
data_df = pd.DataFrame(data)
data_df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [5]:
all_labels = set([label for sublist in data_df['labels'] for label in sublist])
all_labels
label_counts = (data_df['labels'].apply(pd.Series).stack().value_counts())
label_counts

O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

In [139]:
import re 

EMAIL_REGEX = r'\b[A-Za-z0-9]+[.-_]*[A-Za-z0-9]+@(?:gmail|hotmail|yahoo)\.[A-Z|a-z]{2,}\b'
# EMAIL_REGEX = r'\b[A-Za-z0-9]+[.-_]*[A-Za-z0-9]+@[A-Za-z0-9-]+\.[A-Z|a-z]{2,}\b'
PHONE_NUMEBR_REGEX = r'\(\d{1,3}\)\d{3}-\d+(?:x\d+)?|\d{3}\.\d{3}\.\d{1,10}'
# PHONE_NUMEBR_REGEX = r'\(\d{1,3}\)\d{3,14}-\d+(?:x\d+)?|\d{3}\.\d{1,4}\.\d{1,10}'
ADDRESS_REGEX = r'\b\d+\s+[a-zA-Z\s]+(?:\. \d+)[a-zA-Z\s]+(?:\, [A-Z]{2}\s\d{5,6})\b'

NUMBER_REGEX = r'\d+'


def find_email(text):
    return re.findall(re.compile(EMAIL_REGEX), text)

def find_phone_number(text):
    return re.findall(re.compile(PHONE_NUMEBR_REGEX), text)

def find_address(text):
    return re.findall(re.compile(ADDRESS_REGEX), text)

def check_number(token):
    return re.fullmatch(re.compile(NUMBER_REGEX), token)


Filter tokens whose tag contains STREET_ADDRESS

In [138]:
addresses = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if '-STREET_ADDRESS' in label]
addresses

[('591', 'B-STREET_ADDRESS'),
 ('Smith', 'I-STREET_ADDRESS'),
 ('Centers', 'I-STREET_ADDRESS'),
 ('Apt', 'I-STREET_ADDRESS'),
 ('.', 'I-STREET_ADDRESS'),
 ('656', 'I-STREET_ADDRESS'),
 ('\n', 'I-STREET_ADDRESS'),
 ('Joshuamouth', 'I-STREET_ADDRESS'),
 (',', 'I-STREET_ADDRESS'),
 ('RI', 'I-STREET_ADDRESS'),
 ('95963', 'I-STREET_ADDRESS'),
 ('743', 'B-STREET_ADDRESS'),
 ('Erika', 'I-STREET_ADDRESS'),
 ('Bypass', 'I-STREET_ADDRESS'),
 ('Apt', 'I-STREET_ADDRESS'),
 ('.', 'I-STREET_ADDRESS'),
 ('419', 'I-STREET_ADDRESS'),
 ('\n', 'I-STREET_ADDRESS'),
 ('Andreahaven', 'I-STREET_ADDRESS'),
 (',', 'I-STREET_ADDRESS'),
 ('IL', 'I-STREET_ADDRESS'),
 ('54207', 'I-STREET_ADDRESS')]

In [140]:
filtered_addresses = data_df['full_text'].apply(lambda x: find_address(x))
filtered_addresses = filtered_addresses[filtered_addresses.apply(lambda x: x != [])]
filtered_addresses

1103    [591 Smith Centers Apt. 656\nJoshuamouth, RI 9...
1887    [743 Erika Bypass Apt. 419\nAndreahaven, IL 54...
Name: full_text, dtype: object

Filter tokens whose tag is B-EMAIL

In [131]:
emails = [token for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if label == 'B-EMAIL']
emails

['djones@gmail.com',
 'matthew72@hotmail.com',
 'belindarojas@yahoo.com',
 'kennethevans@hotmail.com',
 'agood@gmail.com',
 'agood@gmail.com',
 'hwillis@gmail.com',
 'kellyharrison@gmail.com',
 'kellyharrison@gmail.com',
 'lowetyler@hotmail.com',
 'murraythomas@gmail.com',
 'vmartinez@hotmail.com',
 'sheena82@yahoo.com',
 'nbarker@hotmail.com',
 'nbarker@hotmail.com',
 'nbarker@hotmail.com',
 'johnsondavid@hotmail.com',
 'johnsondavid@hotmail.com',
 'catherine19@hotmail.com',
 'john03@yahoo.com',
 'john03@yahoo.com',
 'lisarose@gmail.com',
 'diazkristen@gmail.com',
 'lisarose@gmail.com',
 'diazkristen@gmail.com',
 'andrew58@gmail.com',
 'boydcynthia@yahoo.com',
 'laurie33@yahoo.com',
 'laurie33@yahoo.com',
 'pauljohnson@hotmail.com',
 'pauljohnson@hotmail.com',
 'jarviscindy@hotmail.com',
 'jarviscindy@hotmail.com',
 'jray@hotmail.com',
 'gday@hotmail.com',
 'gday@hotmail.com',
 'gday@hotmail.com',
 'noah14@hotmail.com',
 'hbrown@yahoo.com']

In [132]:
filtered_emails = data_df['full_text'].apply(lambda x: find_email(x))
filtered_emails = filtered_emails[filtered_emails.apply(lambda x: x != [])]
filtered_emails

24                                     [djones@gmail.com]
86                                [matthew72@hotmail.com]
134     [belindarojas@yahoo.com, kennethevans@hotmail....
171                    [agood@gmail.com, agood@gmail.com]
185                                   [hwillis@gmail.com]
191     [kellyharrison@gmail.com, kellyharrison@gmail....
195                               [lowetyler@hotmail.com]
219                              [murraythomas@gmail.com]
262                               [vmartinez@hotmail.com]
272                                  [sheena82@yahoo.com]
350     [nbarker@hotmail.com, nbarker@hotmail.com, nba...
376     [johnsondavid@hotmail.com, johnsondavid@hotmai...
380                             [catherine19@hotmail.com]
405                  [john03@yahoo.com, john03@yahoo.com]
440     [lisarose@gmail.com, diazkristen@gmail.com, li...
522                                  [andrew58@gmail.com]
550                               [boydcynthia@yahoo.com]
758           

In [135]:
num_emails = filtered_emails.apply(lambda x: len(x)).sum()
print("total number of tokens in data that have B-EMAIL tag: ", len(emails))
print("total number of tokens found by email regex in data: ", num_emails)

total number of tokens in data that have B-EMAIL tag:  39
total number of tokens found by email regex in data:  39


Filter tokens whose tag contains PHONE_NUM

In [230]:
phones = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if '-PHONE_NUM' in label]
phones

[('(', 'B-PHONE_NUM'),
 ('320)202', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('0688x95843', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('223)392', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('2765', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('(', 'B-PHONE_NUM'),
 ('820)913', 'I-PHONE_NUM'),
 ('-', 'I-PHONE_NUM'),
 ('3241x894', 'I-PHONE_NUM'),
 ('410.526.1667', 'B-PHONE_NUM')]

In [136]:
filtered_phone_numbers = data_df['full_text'].apply(lambda x: find_phone_number(x))
filtered_phone_numbers = filtered_phone_numbers[filtered_phone_numbers.apply(lambda x: x != [])]
filtered_phone_numbers

185                                 [(320)202-0688x95843]
219                                       [(223)392-2765]
350     [(820)913-3241x894, (820)913-3241x894, (820)91...
1103                                       [410.526.1667]
Name: full_text, dtype: object

Check '(' token label

In [89]:
paranthesis_tokens = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if token=='(']
paranthesis_labels = {item for sublist in paranthesis_tokens for item in sublist if item != '('}
paranthesis_labels

{'B-PHONE_NUM', 'O'}

Chech '-' token label

In [90]:
hyphen_tokens = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if token=='-']
hyphen_labels = {item for sublist in hyphen_tokens for item in sublist if item != '-'}
hyphen_labels

{'I-PHONE_NUM', 'O'}

Filter number tokens with their labels

In [186]:
numbers = [(token, label) for tokens, labels in zip(data_df['tokens'], data_df['labels']) for token, label in zip(tokens, labels) if check_number(token)]
n  = {item for sublist in numbers for item in sublist if not check_number(item)}
n

{'B-ID_NUM',
 'B-STREET_ADDRESS',
 'I-ID_NUM',
 'I-PHONE_NUM',
 'I-STREET_ADDRESS',
 'O'}